<a href="https://colab.research.google.com/github/Akshitha-Maroju/Image-Stegnography/blob/main/ImageStegnography2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 📦 Install libraries
!pip install gradio pillow

from PIL import Image
import gradio as gr
import io
import os
from datetime import datetime

# 🧊 Encode message into image
def encode_message(image, message):
    try:
        if image is None or message.strip() == "":
            return None, None, "❌ Please upload an image and enter a secret message."

        img = image.convert('RGB')
        encoded = img.copy()
        width, height = img.size
        msg = message + '###'
        binary_msg = ''.join([format(ord(char), '08b') for char in msg])

        if len(binary_msg) > width * height * 3:
            return None, None, "❌ Message too long for this image. Use a larger image."

        data_index = 0
        pixels = list(encoded.getdata())
        new_pixels = []

        for pixel in pixels:
            r, g, b = pixel
            if data_index < len(binary_msg):
                r = (r & ~1) | int(binary_msg[data_index])
                data_index += 1
            if data_index < len(binary_msg):
                g = (g & ~1) | int(binary_msg[data_index])
                data_index += 1
            if data_index < len(binary_msg):
                b = (b & ~1) | int(binary_msg[data_index])
                data_index += 1
            new_pixels.append((r, g, b))

        encoded.putdata(new_pixels)

        # Save image to temporary file
        filename = f"encoded_image_{datetime.now().strftime('%Y%m%d%H%M%S')}.png"
        filepath = f"/tmp/{filename}"
        encoded.save(filepath, format='PNG')

        return encoded, filepath, "✅ Message encoded successfully! Download the image below to decode later."

    except Exception as e:
        return None, None, f"❌ Encoding Error: {str(e)}"

# 🕵️ Decode message from image
def decode_message(image):
    try:
        if image is None:
            return "❌ Please upload an image with a hidden message."

        img = image.convert('RGB')
        pixels = list(img.getdata())
        bits = ''
        message = ''

        for pixel in pixels:
            for color in pixel:
                bits += str(color & 1)
                if len(bits) >= 8:
                    byte = bits[:8]
                    bits = bits[8:]
                    char = chr(int(byte, 2))
                    message += char
                    if message.endswith('###'):
                        return message[:-3]

        return "⚠️ No hidden message found or incorrect image."

    except Exception as e:
        return f"❌ Decoding Error: {str(e)}"

# 🎛️ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🕵️‍♀️ Image Steganography - Hide & Reveal Secret Messages")

    with gr.Tab("🔐 Encode"):
        input_image = gr.Image(type="pil", label="Upload Image (PNG Recommended)")
        secret_text = gr.Textbox(lines=4, label="Secret Message")
        encoded_output = gr.Image(label="🖼️ Encoded Image")
        download_link = gr.File(label="⬇️ Download Encoded Image")
        encode_status = gr.Textbox(label="Status")
        encode_btn = gr.Button("🔐 Encode Message")

        encode_btn.click(
            fn=encode_message,
            inputs=[input_image, secret_text],
            outputs=[encoded_output, download_link, encode_status]
        )

    with gr.Tab("🕵️ Decode"):
        decode_input = gr.Image(type="pil", label="Upload Encoded Image")
        decode_output = gr.Textbox(label="📨 Decoded Secret Message")
        decode_btn = gr.Button("🕵️ Decode Message")

        decode_btn.click(
            fn=decode_message,
            inputs=decode_input,
            outputs=decode_output
        )

# 🚀 Launch Gradio App
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c3400bbc2dc1d2082e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
